# detect faces and check for a mask

In [1]:
import numpy as np
import  os 
import cv2
import matplotlib.pyplot as plt

from mtcnn import MTCNN

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


Using TensorFlow backend.


MTCNN is an implementation of the paper; Zhang, Kaipeng et al. “Joint Face Detection and Alignment Using Multitask Cascaded Convolutional Networks.” IEEE Signal Processing Letters 23.10 (2016): 1499–1503. Crossref. Web.

In [2]:

detector = MTCNN()

## NOTE: Check out how cv2 blobFromImage can be utilized for eliminating illumination changes !!

In [3]:
# load mask detection model
mask_detect_model = load_model('saved_model/my_model_v3.h5')

In [6]:
def check_mask(file_path):
    
    # Load the image from its path
    image_original = cv2.imread(file_path)
    # copy the image
    image = image_original[:]
    detection_results = detector.detect_faces(image)
    
    for i in range(len(detection_results)):
        confidence = detection_results[i]['confidence']
        print("CONFIDENCE: {}".format(confidence))
        # Check if the model is more than 50% confident about finding a face
        if confidence > 0.5:
            # Get the coordinates of the bounding box(location of the face)
            x,y,w,h = detection_results[i]['box']
            print(x,y,w,h)
            
            
            
            # Slice the face from the image and preprocess it
            # pass it to the mask detection model 
            face = image[y:y+h, x:x+w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (160,160))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            print(face.shape)
            
            # put the extracted face to mask detection model 
            # to check for a mask
            (no_mask, mask) = mask_detect_model.predict(face)[0]
            print('MASK, NO MASK ', mask, no_mask)
            
            # choose the label and the color of the bounding box
            # according to the prediction
            label = ''
            color = (0,0,0)
            if mask > no_mask:
                label = 'MASK'
                color = (0,255,0)
            else:
                label = 'NO MASK'
                color = (0,0,255)
            
            cv2.putText(image, label, (x, y-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (x,y),(x+w, y+h), color, 2)
    cv2.imshow("Output", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [7]:
check_mask('test_images/cagri.jpg')


CONFIDENCE: 0.9999349117279053
307 340 221 272
(1, 160, 160, 3)
MASK, NO MASK  0.018040944 0.98195904
CONFIDENCE: 0.9699272513389587
1012 315 125 151
(1, 160, 160, 3)
MASK, NO MASK  0.06726437 0.9327356
